# Baseline models

# 2. ViLT

### Setup

In [3]:
!pip install transformers accelerate bert-score scikit-learn timeout-decorator pandas pillow torch

In [14]:
# Cell 2: Imports and Setup
import os
import glob
import pandas as pd
import torch
from PIL import Image
from timeout_decorator import timeout, TimeoutError
from sklearn.metrics import accuracy_score, f1_score
from bert_score import score as bertscore_score
from transformers import ViltProcessor, ViltForQuestionAnswering
from tqdm import tqdm

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on", device)

Running on cuda


### Load and Concatenate Curated Dataset (S1–S6)

In [5]:
base_path = "/kaggle/input/vr-dataset/dataset_curated"
subfolders = [f"S{i}" for i in range(1, 7)]
all_dfs = []

for folder in subfolders:
    csv_path = os.path.join(base_path, folder, f"{folder}_qa_data.csv")
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path, header=None, names=["image_path", "question", "answer"])
        df["image_path"] = df["image_path"].apply(
            lambda p: os.path.join(base_path, os.path.normpath(p).split("dataset_curated/")[-1])
        )
        df = df[df["image_path"].apply(os.path.exists)].reset_index(drop=True)
        all_dfs.append(df)
        print(f"Loaded {len(df)} examples from {csv_path}")
    else:
        print(f"CSV not found: {csv_path}")

df_all = pd.concat(all_dfs).reset_index(drop=True)
print(f"\nTotal QA pairs loaded: {len(df_all)}")

Loaded 14366 examples from /kaggle/input/vr-dataset/dataset_curated/S1/S1_qa_data.csv
Loaded 14358 examples from /kaggle/input/vr-dataset/dataset_curated/S2/S2_qa_data.csv
Loaded 14367 examples from /kaggle/input/vr-dataset/dataset_curated/S3/S3_qa_data.csv
Loaded 14366 examples from /kaggle/input/vr-dataset/dataset_curated/S4/S4_qa_data.csv
Loaded 14387 examples from /kaggle/input/vr-dataset/dataset_curated/S5/S5_qa_data.csv
Loaded 14376 examples from /kaggle/input/vr-dataset/dataset_curated/S6/S6_qa_data.csv

Total QA pairs loaded: 86220


### Initialize ViLT Processor & Model

In [6]:

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

model.to(device)
model.eval()


ViltForQuestionAnswering(
  (vilt): ViltModel(
    (embeddings): ViltEmbeddings(
      (text_embeddings): TextEmbeddings(
        (word_embeddings): Embedding(30522, 768)
        (position_embeddings): Embedding(40, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (patch_embeddings): ViltPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
      )
      (token_type_embeddings): Embedding(2, 768)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViltEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViltLayer(
          (attention): ViltAttention(
            (attention): ViltSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=76

### Define Inference Function with Timeout

In [7]:
@timeout(60)  # max 60s per sample
def predict_answer(image_path: str, question: str) -> str:
    try:
        # 1. Load and preprocess
        image = Image.open(image_path).convert("RGB")
        inputs = processor(image, question, return_tensors="pt")
        # **NEW**: move inputs to GPU if model is on GPU
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # 2. Forward pass
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits

        # 3. Decode
        pred_id = logits.argmax(-1).item()
        answer = model.config.id2label[pred_id]
        return answer.split()[0].strip().lower()

    except Exception as e:
        # Print the first few errors to see what’s wrong
        if predict_answer.err_count < 5:
            print(f"[Error] {type(e).__name__}: {e}")
        predict_answer.err_count += 1
        return ""
# initialize counter
predict_answer.err_count = 0

### Run Inference & Collect Predictions

In [15]:
# Main evaluation loop with 1,000-sample checkpoints
# Prepare for resuming
# Paths for checkpointing
pred_path    = "/kaggle/working/vilt_vqa_predictions.csv"
metrics_path = "/kaggle/working/vilt_vqa_metrics_summary.csv"


start_idx = 0
if os.path.exists(pred_path):
    existing = pd.read_csv(pred_path)
    start_idx = len(existing)
    print("Resuming from index", start_idx)

# Buffers
buffer_results = []
buffer_y_true  = []
buffer_y_pred  = []

for idx, row in tqdm(df_all.iterrows(), total=len(df_all), desc="Evaluating"):
    if idx < start_idx:
        continue

    img_path = row["image_path"]
    question = row["question"]
    true_ans = str(row["answer"]).strip().lower()

    pred = predict_answer(img_path, question)
    pred = pred or ""  # ensure string
    pred = pred.lower()

    buffer_results.append({
        "img_path": img_path,
        "question": question,
        "true_answer": true_ans,
        "predicted_answer": pred
    })
    buffer_y_true.append(true_ans)
    buffer_y_pred.append(pred)

    # checkpoint every 1,000 or at end
    if (idx + 1) % 1000 == 0 or (idx + 1) == len(df_all):
        # 1) save predictions chunk
        chunk_df = pd.DataFrame(buffer_results)
        mode    = "w" if not os.path.exists(pred_path) else "a"
        header  = not os.path.exists(pred_path)
        chunk_df.to_csv(pred_path, mode=mode, index=False, header=header)

        # 2) compute interim metrics
        acc = accuracy_score(buffer_y_true, buffer_y_pred)
        f1m = f1_score(buffer_y_true, buffer_y_pred, average="macro", zero_division=0)
        # BERTScore
        P, R, F1b = bertscore_score(buffer_y_pred, buffer_y_true, lang="en")
        bert = F1b.mean().item()

        metrics_df = pd.DataFrame([{
            "step": idx+1,
            "accuracy": acc,
            "f1_macro": f1m,
            "bert_score": bert
        }])
        m_mode   = "w" if not os.path.exists(metrics_path) else "a"
        m_header = not os.path.exists(metrics_path)
        metrics_df.to_csv(metrics_path, mode=m_mode, index=False, header=m_header)

        print(f"Checkpoint @ {idx+1}: acc={acc:.4f}, f1={f1m:.4f}, bert={bert:.4f}")

        # clear buffers
        buffer_results.clear()
        buffer_y_true.clear()
        buffer_y_pred.clear()

print("Evaluation complete.")

Evaluating:   1%|          | 997/86220 [00:25<37:14, 38.15it/s]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating:   1%|          | 1005/86220 [00:28<3:54:19,  6.06it/s]

Checkpoint @ 1000: acc=0.2950, f1=0.0808, bert=0.9749


Evaluating:   2%|▏         | 1996/86220 [00:56<43:01, 32.63it/s]  Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating:   2%|▏         | 2004/86220 [00:57<2:17:55, 10.18it/s]

Checkpoint @ 2000: acc=0.3020, f1=0.0825, bert=0.9767


Evaluating:   3%|▎         | 2998/86220 [01:27<42:20, 32.76it/s]  Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating:   3%|▎         | 3006/86220 [01:28<2:18:45,  9.99it/s]

...

Checkpoint @ 32000: acc=0.2960, f1=0.0725, bert=0.9748


Evaluating:  38%|███▊      | 32999/86220 [16:06<23:58, 36.99it/s]  Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating:  38%|███▊      | 33007/86220 [16:08<1:32:57,  9.54it/s]

Checkpoint @ 33000: acc=0.2860, f1=0.0631, bert=0.9728


Evaluating:  39%|███▉      | 33764/86220 [16:29<25:37, 34.13it/s]  


KeyboardInterrupt: 

### Compute Evaluation Metrics

In [16]:
y_true = df_all["answer"].astype(str).str.lower().tolist()
y_pred = df_all["predicted_answer"].astype(str).str.lower().tolist()

accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="macro")

# BERTScore: rescaled against baseline
P, R, F1_bert = bertscore_score(y_pred, y_true, lang="en", rescale_with_baseline=True)
bert_score = F1_bert.mean().item()

print(f"Accuracy:  {accuracy:.4f}")
print(f"F1 Macro:  {f1:.4f}")
print(f"BERTScore: {bert_score:.4f}")


KeyError: 'predicted_answer'